<a href="https://colab.research.google.com/github/SRARNAB7/HDS_5230_07_Arnab/blob/main/Week%2009/Week_09_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Import Modules
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.metrics import confusion_matrix
import sklearn
from sklearn import datasets

In [2]:
## Set default figure size to be larger
## this may only work in matplotlib 2.0+!
matplotlib.rcParams['figure.figsize'] = [10.0,6.0]
## Enable multiple outputs from jupyter cells
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
## Get Version information
print(sys.version)
print("Pandas version: {0}".format(pd.__version__))
print("Matplotlib version: {0}".format(matplotlib.__version__))
print("Numpy version: {0}".format(np.__version__))
print("SciKitLearn version: {0}".format(sklearn.__version__))

3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Pandas version: 2.2.2
Matplotlib version: 3.10.0
Numpy version: 2.0.2
SciKitLearn version: 1.6.1


Check your working directory
Set your working directory to make paths easier :)

In [4]:
# Working Directory
import os
print("My working directory:\n" + os.getcwd())
# Set Working Directory
os.chdir(".")
print("My new working directory:\n" + os.getcwd())

My working directory:
/content
My new working directory:
/content


Patient Mortality Dataset
We will use a dataset with a binary outcome of mortality as a motivating example.

This is a dataset of patients demographics and disease status, with mortality indicated. The dataset is here:

data\healthcare\patientAnalyticFile.csv

In practice, you most likely would have created a dataset like this from multiple other files after cleaning, reshaping, and joining them.

You can generalize this setup to any situation with a binary outcome, such as estimating the probability of a customer filing a warranty claim, or the probability of a transaction being fraudulent.

We will first import this dataset and examine the potential variables to use in our classification algorithm.

In [5]:
## Set print limits
pd.options.display.max_rows = 10
## Import Data
df_patient = \
 pd.read_csv('./PatientAnalyticFile.csv')
df_patient

,PatientID,DateOfBirth,Gender,Race,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,...,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,First_Appointment_Date,Last_Appointment_Date,DateOfDeath
0,1,1962-02-27,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2013-04-27,2018-06-01,NaN
1,2,1959-08-18,male,white,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2005-11-30,2008-11-02,2008-11-02
2,3,1946-02-15,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2011-11-05,2015-11-13,NaN
3,4,1979-07-27,female,white,0,0,0,0,0,1,...,0,0,0,0,0,0,0,2010-03-01,2016-01-17,2016-01-17
4,5,1983-02-19,female,hispanic,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2006-09-22,2018-06-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,1997-12-19,female,other,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2008-06-14,2018-06-01,NaN
19996,19997,1984-03-31,female,white,0,0,0,0,0,0,...,0,1,0,0,1,0,0,2007-04-24,2018-06-01,NaN
19997,19998,1993-07-04,female,white,0,0,0,0,0,0,...,0,0,1,0,1,0,0,2010-10-16,2018-06-01,NaN
19998,19999,1984-04-17,male,other,0,0,0,0,0,0,...,0,0,0,0,1,0,0,2015-01-04,2018-06-01,NaN


We need to make a variable to indicate mortality. We can do that based on the abscence of 'date of death':

In [6]:
# Create mortality variable
df_patient['mortality'] = \
    np.where(df_patient['DateOfDeath'].isnull(),
             0,1)
# Examine
df_patient['mortality']

,mortality
0,0
1,1
2,0
3,1
4,0
...,...
19995,0
19996,0
19997,0
19998,0


In [7]:
df_patient['mortality'].describe()

,mortality
count,20000.000000
mean,0.354700
std,0.478434
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [8]:
df_patient.describe()

,PatientID,Myocardial_infarction,Congestive_heart_failure,Peripheral_vascular_disease,Stroke,Dementia,Pulmonary,Rheumatic,Peptic_ulcer_disease,LiverMild,...,Cancer,LiverSevere,Metastatic_solid_tumour,HIV,Obesity,Depression,Hypertension,Drugs,Alcohol,mortality
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,...,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,0.045600,0.043450,0.023950,0.028650,0.031400,0.072650,0.012300,0.009650,0.009250,...,0.050450,0.051450,0.033150,0.006450,0.163450,0.106300,0.302900,0.040050,0.079750,0.354700
std,5773.647028,0.208621,0.203873,0.152897,0.166825,0.174401,0.259568,0.110224,0.097762,0.095733,...,0.218877,0.220919,0.179033,0.080054,0.369785,0.308229,0.459524,0.196081,0.270913,0.478434
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5000.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10000.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,15000.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
max,20000.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
df_patient.dtypes

,0
PatientID,int64
DateOfBirth,object
Gender,object
Race,object
Myocardial_infarction,int64
...,...
Alcohol,int64
First_Appointment_Date,object
Last_Appointment_Date,object
DateOfDeath,object


We should change date of birth to be an actual date and calculate age if we want to include it in the model:

In [10]:
# Convert dateofBirth to date
df_patient['DateOfBirth'] = \
    pd.to_datetime(df_patient['DateOfBirth'])
# Calculate age in years as of 2015-01-01
df_patient['Age_years'] = \
    ((pd.to_datetime('2015-01-01') - df_patient['DateOfBirth']).dt.days/365.25)
df_patient['Age_years'].describe()

,Age_years
count,20000.000000
mean,47.247474
std,18.145086
min,15.753593
25%,31.733744
50%,47.099247
75%,62.924025
max,78.743326


Use Patsy to Create the Model Matrices
We typically start out with a pandas dataframe for manipulation purposes, then we will use this dataframe as the input to the machine learning library. I created a pandas dataframe above to replicate this process. We will use the dmatrices function from the patsy library to easily generate the design matrices for the machine learning algorithms representing the inputs. THis handles the following:

drops rows with missing data
construct one-hot encoding for categorical variables
optionally adds constant intecercept

In [11]:
df_patient.columns

Index(['PatientID', 'DateOfBirth', 'Gender', 'Race', 'Myocardial_infarction',
       'Congestive_heart_failure', 'Peripheral_vascular_disease', 'Stroke',
       'Dementia', 'Pulmonary', 'Rheumatic', 'Peptic_ulcer_disease',
       'LiverMild', 'Diabetes_without_complications',
       'Diabetes_with_complications', 'Paralysis', 'Renal', 'Cancer',
       'LiverSevere', 'Metastatic_solid_tumour', 'HIV', 'Obesity',
       'Depression', 'Hypertension', 'Drugs', 'Alcohol',
       'First_Appointment_Date', 'Last_Appointment_Date', 'DateOfDeath',
       'mortality', 'Age_years'],
      dtype='object')

In [12]:
## Create formula for all variables in model
vars_remove = ['PatientID','First_Appointment_Date','DateOfBirth',
               'Last_Appointment_Date','DateOfDeath','mortality']
vars_left = set(df_patient.columns) - set(vars_remove)
formula = "mortality ~ " + " + ".join(vars_left)
formula

'mortality ~ Alcohol + Dementia + Diabetes_with_complications + Pulmonary + LiverSevere + Stroke + Drugs + Rheumatic + Gender + Age_years + Peripheral_vascular_disease + Metastatic_solid_tumour + Paralysis + Obesity + Peptic_ulcer_disease + Race + Cancer + Myocardial_infarction + Depression + LiverMild + Congestive_heart_failure + HIV + Diabetes_without_complications + Hypertension + Renal'

Next, fit a series of logistic regression models, without regularization. Each model should use the same set of predictors (all of the relevant predictors in the dataset) and should use the entire dataset, rather than a fraction of it. Use a randomly chosen 80% proportion of observations for training and the remaining for checking the generalizable performance (i.e., performance on the holdout subset). Be sure to ensure that the training and holdout subsets are identical across all models. Each model should choose a different solver.

In [13]:
Y, X = dmatrices(formula, df_patient)

In [14]:
Y

DesignMatrix with shape (20000, 1)
  mortality
          0
          1
          0
          1
          0
          0
          0
          0
          0
          0
          0
          0
          1
          0
          0
          0
          1
          1
          0
          0
          1
          1
          0
          0
          0
          0
          0
          0
          0
          0
  [19970 rows omitted]
  Terms:
    'mortality' (column 0)
  (to view full data, use np.asarray(this_obj))

In [15]:
X

DesignMatrix with shape (20000, 28)
  Columns:
    ['Intercept',
     'Gender[T.male]',
     'Race[T.hispanic]',
     'Race[T.other]',
     'Race[T.white]',
     'Alcohol',
     'Dementia',
     'Diabetes_with_complications',
     'Pulmonary',
     'LiverSevere',
     'Stroke',
     'Drugs',
     'Rheumatic',
     'Age_years',
     'Peripheral_vascular_disease',
     'Metastatic_solid_tumour',
     'Paralysis',
     'Obesity',
     'Peptic_ulcer_disease',
     'Cancer',
     'Myocardial_infarction',
     'Depression',
     'LiverMild',
     'Congestive_heart_failure',
     'HIV',
     'Diabetes_without_complications',
     'Hypertension',
     'Renal']
  Terms:
    'Intercept' (column 0)
    'Gender' (column 1)
    'Race' (columns 2:5)
    'Alcohol' (column 5)
    'Dementia' (column 6)
    'Diabetes_with_complications' (column 7)
    'Pulmonary' (column 8)
    'LiverSevere' (column 9)
    'Stroke' (column 10)
    'Drugs' (column 11)
    'Rheumatic' (column 12)
    'Age_years' (column 1

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, np.ravel(Y), test_size=0.2, random_state=42)

Confirm the Output Dimensions
We can confirm the dimensions of the data are the same within test and train
The proportion should also be close to the test_size argument.

In [17]:
## Confirm dimensions
X_train.shape

(16000, 28)

In [18]:
X_test.shape

(4000, 28)

In [19]:
y_train.shape

(16000,)

In [20]:
y_test.shape

(4000,)

Logistic Regression Models with Different Solvers that are not generalized

In [21]:
#solvers = ['lbfgs', 'newton-cg', 'newton-cholesky', 'sag', 'saga']

In [22]:
## Import the linear model
from sklearn import linear_model

## Define logistic regression model without regularization
clf = linear_model.LogisticRegression(
    fit_intercept=True,       # Include intercept
    penalty= None,           # No regularization
    solver='lbfgs',           # Solver (change to 'sag', 'saga', etc. as needed)
    max_iter=1000,            # Increase if convergence warning appears
    random_state=42           # For reproducibility
)

## Fit the model on the training data
clf.fit(X_train, y_train)

LogisticRegression(max_iter=1000, penalty=None, random_state=42)

In [23]:
## Make predictions on training dataset
## training error?
clf.predict(X_train)

array([1., 0., 0., ..., 1., 0., 0.])

In [24]:
## can also predict probabilities
clf.predict_proba(X_train)

array([[0.3135028 , 0.6864972 ],
       [0.80862981, 0.19137019],
       [0.75100615, 0.24899385],
       ...,
       [0.45239743, 0.54760257],
       [0.93714714, 0.06285286],
       [0.95138866, 0.04861134]])

Model Summaries
We can extract the model coefficients with the .coef_ attribute.

In [25]:
## Get coefficients
clf.coef_
clf.coef_.shape

array([[-2.12697649,  0.02506059, -0.0686817 , -0.10013736, -0.00678888,
         0.79068287,  0.0156925 ,  0.15443059,  0.63154985,  0.97892936,
        -0.11207666,  0.33467554, -0.30506971,  0.06503364,  1.13509639,
         0.03131625,  0.14207435,  0.3080959 , -0.03757431,  0.65018811,
         0.86298698,  0.50502501,  0.14850754,  0.47092241,  0.14993887,
        -0.002132  , -0.03135645, -0.06491718]])

(1, 28)

Assessing the Model Score (accuracy)
The concept of error is a bit different for a binary outcome than the continuous case.
We can construct error to be a function of the number of incorrect predictions.

In [26]:
## get mean accuracy
clf.score(X_train,y_train)

0.747875

In this case, our model is accurate about 74% of the time! We can understand what that means by looking at the predictions against the actual outcomes. This is called a confusion matrix.

In [27]:
## get confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,
                 clf.predict(X_train))

array([[8745, 1599],
       [2435, 3221]])

In [28]:
## get classification metrics
print(sklearn.metrics.classification_report(y_train,
                                            clf.predict(X_train)))

              precision    recall  f1-score   support

         0.0       0.78      0.85      0.81     10344
         1.0       0.67      0.57      0.61      5656

    accuracy                           0.75     16000
   macro avg       0.73      0.71      0.71     16000
weighted avg       0.74      0.75      0.74     16000



In [34]:
## get accuracy
sklearn.metrics.accuracy_score(y_train,
                               clf.predict(X_train))

0.747875

Keep Track of Scores Across Models
I am going to write a small function that will print the scores from a dict so we can compare the models. I will store the model scores in the dict as well.

In [35]:
## Create dict to store all these results:
result_scores = {}
## Score the Model on Training and Testing Set
result_scores['Logistic'] = \
            (sklearn.metrics.accuracy_score(y_train,clf.predict(X_train)),
             sklearn.metrics.accuracy_score(y_test,clf.predict(X_test)))

In [36]:
## Create Function to Print Results
def get_results(x1):
    print("\n{0:20}   {1:4}    {2:4}".format('Model','Train','Test'))
    print('-------------------------------------------')
    for i in x1.keys():
        print("{0:20}   {1:<6.4}   {2:<6.4}".format(i,x1[i][0],x1[i][1]))

In [37]:
get_results(result_scores)


Model                  Train    Test
-------------------------------------------
Logistic               0.7479   0.7355
